A minimal approach to use transformers using the simpletransformers package (https://github.com/ThilinaRajapakse/simpletransformers/) that allows for:
1. Easy switching between different pre-trained models
2. Simple hyperparameter tuning
3. Basic k-fold cross validation
4. Zero pre-processing of data

However, language model fine-tuning is not available with this method.

Updates:
12 Jan 2020 - Update to set random seed to ensure deterministic prediction.

In [ ]:
!pip install simpletransformers

In [ ]:
import os, re, string
import random

import numpy as np
import pandas as pd
import sklearn

import torch

from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
seed = 1337

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Zero pre-processing of training data
# I have tried doing some preprcessing/cleaning but the result 
# does not seem significant
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
train_data = train_data[['text', 'target']]

# Check default value of model and tweak

In [ ]:
# Using 'bert-large-uncased' here. For a list of other models, please refer to 
# https://github.com/ThilinaRajapakse/simpletransformers/#current-pretrained-models 
bert_uncased = ClassificationModel('bert', 'bert-large-uncased') 

# Print out all the default arguments for reference
bert_uncased.args

In [ ]:
# This is where we can tweak based on the default arguments above
custom_args = {'fp16': False, # not using mixed precision 
               'train_batch_size': 4, # default is 8
               'gradient_accumulation_steps': 2,
               'do_lower_case': True,
               'learning_rate': 1e-05, # using lower learning rate
               'overwrite_output_dir': True, # important for CV
               'num_train_epochs': 2} # default is 1

# 5-Fold CV

In [ ]:
n=5
kf = KFold(n_splits=n, random_state=seed, shuffle=True)
results = []

for train_index, val_index in kf.split(train_data):
    train_df = train_data.iloc[train_index]
    val_df = train_data.iloc[val_index]
    
    model = ClassificationModel('bert', 'bert-base-uncased', args=custom_args) 
    model.train_model(train_df)
    result, model_outputs, wrong_predictions = model.eval_model(val_df, acc=sklearn.metrics.accuracy_score)
    print(result['acc'])
    results.append(result['acc'])

In [ ]:
for i, result in enumerate(results, 1):
    print(f"Fold-{i}: {result}")
    
print(f"{n}-fold CV accuracy result: Mean: {np.mean(results)} Standard deviation:{np.std(results)}")

# Full Training

In [ ]:
model = ClassificationModel('bert', 'bert-base-uncased', args=custom_args) 
model.train_model(train_data)

# Predict

In [ ]:
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
predictions, raw_outputs = model.predict(test_data['text'])

sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] = predictions
sample_submission.to_csv("submission.csv", index=False)